In [20]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric, ContextualPrecisionMetric, ContextualRecallMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset
import subprocess
from deepeval.models.base_model import DeepEvalBaseLLM
from groq import Groq
import os
import traceback
import json
os.environ["DEEPEVAL_RESULTS_FOLDER"] = "D:\Jupyter Notebooks\Evaluation\evaluation_results"
dataset_path="D:\Jupyter Notebooks\Evaluation\generated_testset\deepeval_dataset_formatted.json"
STATUS_FILE = "D:\\Jupyter Notebooks\\Evaluation\\evaluation_results\\evaluation_status.json"

In [ ]:
GROQ_API_KEY = "***"

In [32]:
class GroqLLM(DeepEvalBaseLLM):
    def __init__(self, api_key: str, model_name: str):
        self.api_key = api_key
        self.model_name = model_name
        self.client = Groq(api_key=self.api_key)

    def load_model(self):
        return self.client

    def generate(self, prompt: str) -> str:
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=1000,
            response_format={ "type": "json_object" }
        )
        return response.choices[0].message.content.strip()

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self) -> str:
        return f"groq-{self.model_name}"


In [36]:
if not os.path.exists(STATUS_FILE):
    with open(STATUS_FILE, 'w', encoding='utf-8') as f:
        json.dump({"success_results": [], "fail_results": []}, f, indent=2)

with open(STATUS_FILE, 'r', encoding='utf-8') as f:
    status_data = json.load(f)

def save_state():
    with open(STATUS_FILE, 'w', encoding='utf-8') as f:
        json.dump(status_data, f, indent=2, ensure_ascii=False)

In [ ]:
with open(dataset_path, 'r', encoding='utf-8') as f:
    test_data = json.load(f)

success_results = []
fail_results = []
groq_model = GroqLLM(api_key=GROQ_API_KEY, model_name="llama3-70b-8192")

metrics = [
    AnswerRelevancyMetric(model=groq_model),
    ContextualPrecisionMetric(model=groq_model),
    ContextualRecallMetric(model=groq_model),
    ContextualRelevancyMetric(model=groq_model)
]

for entry in test_data:
    if entry["id"] in status_data["success_results"]:
        continue
    try:
        test_case = LLMTestCase(
            input=entry["query"],
            actual_output=entry["actual_output"],
            expected_output=entry["expected_output"],
            context=entry["context"],
            retrieval_context=entry["retrieval_context"]
        )

        dataset = EvaluationDataset(test_cases=[test_case])
        evaluate(dataset, metrics=metrics)
        status_data["success_results"].append(entry["id"])
        save_state() 
    except Exception as e:
        error_message = str(e)        
        status_data["fail_results"].append((entry["id"], error_message))
        save_state()        
        print(f"Error al evaluar {entry['id']}: {error_message}")
        if "Limit" in error_message and "TPD" in error_message:
            print("Se alcanzó el límite de tokens por día. Deteniendo el proceso.")
            break

✨ You're running DeepEval's latest Answer Relevancy Metric! (using groq-llama3-70b-8192, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using groq-llama3-70b-8192, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using groq-llama3-70b-8192, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using groq-llama3-70b-8192, strict=False, 
async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████████████████████████|100% (1/1) [Time Taken: 00:24, 24.83s/test case]

...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using groq-llama3-70b-8192, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Precision Metric! (using groq-llama3-70b-8192, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Recall Metric! (using groq-llama3-70b-8192, strict=False, 
async_mode=True)...

✨ You're running DeepEval's latest Contextual Relevancy Metric! (using groq-llama3-70b-8192, strict=False, 
async_mode=True)...

Evaluating 1 test case(s) in parallel: |                                  |  0% (0/1) [Time Taken: 00:00, ?test case/s]